In [36]:
%pip -q install google-genai

In [37]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [38]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [39]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura não tem uma data fixa para a próxima Imersão IA com Google Gemini. A melhor forma de saber quando a próxima edição será lançada é:

*   **Acompanhar as redes sociais e o site da Alura:** A Alura costuma anunciar seus eventos e cursos em suas redes sociais (LinkedIn, Instagram, etc.) e em seu site.
*   **Assinar a newsletter da Alura:** Assim, você receberá as novidades e anúncios diretamente no seu e-mail.
*   **Ficar de olho na página da Imersão IA:** Mesmo que não tenha uma data no momento, a página da Imersão IA no site da Alura será atualizada quando houver novas informações.

In [40]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura acontecerá entre os dias 12 e 16 de maio de 2025. A imersão é gratuita e totalmente online, sem pré-requisitos. As inscrições podem ser feitas até 11 de maio.

A Imersão IA oferece cinco aulas onde você vai aprender a criar prompts poderosos, automatizar tarefas e desenvolver um sistema de agentes autônomos. Ao final da imersão, você receberá um certificado Alura + Google.


In [41]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['Alura Imersão IA com Google Gemini']
Páginas utilizadas na resposta: alura.com.br, tecmundo.com.br



In [42]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

In [43]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [44]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [45]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [46]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
        Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ele não seja tão relevante assim
        e pode ser substituído por outro que tenha mais.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [47]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de
        lançamentos mais recentes e relevantes buscador, você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano sobre
        quais são os pontos mais relevantes que poderíamos abordar em um post sobre
        cada um deles. Você também pode usar o (google_search) para encontrar mais
        informações sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas
        e retornar esse tema, seus pontos mais relevantes, e um plano com os assuntos
        a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [48]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [49]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [50]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}")

    lancamentos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    plano_de_post = agente_planejador(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(plano_de_post))
    print("--------------------------------------------------------------")

    rascunho_de_post = agente_redator(topico, plano_de_post)
    print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(rascunho_de_post))
    print("--------------------------------------------------------------")

    post_final = agente_revisor(topico, rascunho_de_post)
    print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
    display(to_markdown(post_final))
    print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: bebes reborn


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-74' coro=<AsyncClient.aclose() done, defined at /usr/local/lib/python3.11/dist-packages/httpx/_client.py:1978> exception=RuntimeError('Event loop is closed')>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/httpx/_client.py", line 1985, in aclose
    await self._transport.aclose()
  File "/usr/local/lib/python3.11/dist-packages/httpx/_transports/default.py", line 406, in aclose
    await self._pool.aclose()
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection_pool.py", line 353, in aclose
    await self._close_connections(closing_connections)
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection_pool.py", line 345, in _close_connections
    await connection.aclose()
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection.py", line 173, in aclose
    await self._connection.aclose()
  File "/usr/local/lib/pytho

Maravilha! Vamos então criar o post sobre novidades em bebes reborn

--- 📝 Resultado do Agente 1 (Buscador) ---



> Aqui estão algumas notícias recentes sobre bebês reborn:
> 
> 
> Com base nas notícias mais recentes sobre bebês reborn, aqui estão alguns lançamentos e tendências relevantes:
> 
> 1.  **Realismo Ultra em Alta:** Em 2025, a busca por modelos ultrarrealistas de bebês reborn está mais forte do que nunca. As inovações incluem detalhes como textura da pele, movimentos leves e expressões faciais detalhadas que os fazem parecer prestes a abrir os olhos ou sorrir.
> 2.  **Materiais de Alta Qualidade:** Os bebês reborn mais realistas de 2025 são feitos com materiais de alta qualidade, como silicone que imita a pele humana. Técnicas avançadas de pintura e cabelo implantado à mão também contribuem para o realismo.
> 3.  **Modelos Populares:**
>     *   **VXN Menina Princesa:** Considerada uma das melhores bebês reborn de 2025.
>     *   **Shiny Toys Laura Baby Mini Valentina:** Destacada pelo custo-benefício.
>     *   **Reborn Realista Girafinha:** Feita de silicone, proporcionando um toque realista.
> 4.  **Opções de Compra:** Diversas opções estão disponíveis para compra online, incluindo modelos de silicone e vinil, com diferentes características e preços. Lojas como "Reborn Bebe" oferecem uma variedade de bebês reborn com promoções e lançamentos.
> 5.  **Bebês Reborn Interativos:** Há modelos que fazem cocô e xixi, acompanhados de enxoval, proporcionando uma experiência mais completa e realista.


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Planejador) ---



...

--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Redator) ---



> **Rascunho do Post para Instagram:**
> 
> ✨ Bebês Reborn 2025: Prepare-se para o Realismo MÁXIMO e a Interatividade INACREDITÁVEL! ✨
> 
> Já ouviu falar dos bebês reborn? 🤔 Eles são a sensação do momento e, em 2025, estão mais realistas do que nunca! 😍
> 
> Imagine ter um bebê que parece de verdade, com textura de pele ultra realista, movimentos leves e até expressões faciais super detalhadas. É o realismo ultra que chegou para impressionar! 😉
> 
> E não para por aí! Os materiais são de altíssima qualidade, como o silicone que imita a pele humana, e as técnicas de pintura e implantação de cabelo são feitas à mão, garantindo um visual INCRÍVEL.
> 
> 🌟 Destaque para os modelos que estão bombando: VXN Menina Princesa, Shiny Toys Laura Baby Mini Valentina e Reborn Realista Girafinha! Cada um com um charme especial e detalhes que encantam.
> 
> 🙊 E a novidade que vai te deixar de boca aberta: agora os bebês reborn FAZEM COCÔ E XIXI! 😱 Acompanham até enxoval completo para a brincadeira ficar ainda mais real.
> 
> Quer saber onde encontrar o seu bebê reborn perfeito? 👀 Existem diversas lojas online e físicas, como a "Reborn Bebe". Mas ATENÇÃO: compre sempre de lojas confiáveis para garantir a qualidade e a segurança do produto, combinado? 😉
> 
> Os bebês reborn são pura magia e encantam crianças e adultos. 🥰 É um mundo de detalhes e realismo que você precisa conhecer!
> 
> Gostou? Marque um amigo que também vai AMAR essa novidade! 👇
> 
> #BebesReborn #RealismoUltra #Maternidade #BonecasRealistas
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Revisor) ---



> O rascunho precisa de alguns ajustes para ficar mais atraente e informativo para o público do Instagram:
> 
> **Problemas e Sugestões:**
> 
> *   **Excesso de Ênfase:** Muitas palavras em maiúsculo e emojis podem cansar o leitor. Modere o uso para não parecer apelativo demais.
> *   **Informação Desorganizada:** O texto parece um pouco confuso. Reorganize as informações para criar um fluxo mais lógico.
> *   **Falta de Chamada para Ação Clara:** Incentive mais o engajamento, além de marcar amigos.
> *   **Hashtags:** Poderiam ser mais específicas e populares.
> 
> **Rascunho Revisado:**
> 
> ✨ Bebês Reborn 2025: Realismo e Interatividade que Surpreendem! ✨
> 
> Já ouviu falar dos bebês reborn? 🤔 A febre do momento está ainda mais realista em 2025! 😍
> 
> Imagine um bebê que parece de verdade: textura de pele ultra realista, movimentos suaves e expressões faciais super detalhadas. Demais, né? 😉
> 
> A qualidade impressiona! Feitos com silicone que imita a pele humana, pintura e cabelo implantados à mão, um visual incrível e único.
> 
> 🌟 Modelos que estão bombando: VXN Menina Princesa, Shiny Toys Laura Baby Mini Valentina e Reborn Realista Girafinha! Qual o seu preferido?
> 
> 🙊 Prepare-se: agora os bebês reborn FAZEM COCÔ E XIXI! 😱 E acompanham um enxoval completo pra brincadeira ficar super real.
> 
> Onde encontrar o seu? 👀 Em lojas online e físicas como a "Reborn Bebe". Mas atenção: compre sempre em lojas confiáveis pra garantir a qualidade e a segurança, ok? 😉
> 
> Os bebês reborn são pura magia e encantam! 🥰 Um mundo de detalhes e realismo que você precisa conhecer!
> 
> Curtiu? Deixe um comentário com o seu modelo preferido e marque um amigo que também vai amar! 👇
> 
> #BebesReborn #BonecasReborn #Realismo #MaternidadeReal #BrinquedosRealistas #Colecionaveis #BabyReborn #Novidades #MiniValentina #VXNMeninaPrincesa #ShinyToysLauraBaby
> 
> **Justificativas das Mudanças:**
> 
> *   **Tom:** Mais entusiasmado e menos "marketeiro".
> *   **Organização:** Informações dispostas de forma mais clara e convidativa.
> *   **Hashtags:** Adição de hashtags mais específicas e populares para aumentar o alcance.
> *   **Chamada para Ação:** Incentivo a comentários e marcação de amigos para aumentar o engajamento.
> *   **Remoção de Excessos:** Eliminação de repetições e moderação no uso de maiúsculas e emojis.
> 
> Com essas mudanças, o post deve performar melhor e atrair mais atenção do público-alvo no Instagram.


--------------------------------------------------------------
